#**Import Modules**

In [0]:
from google.colab import files

In [0]:
import openpyxl

In [0]:
import time

In [0]:
from os import listdir
from os.path import isfile, join

#**Mount Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [0]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/  Untitled


#**Modify FIles**

In [0]:
DATAPATH = "./Data/raw/Korean/"

In [0]:
# fetch file names
raw_files = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]

In [0]:
for raw in raw_files:
    wb = openpyxl.load_workbook(DATAPATH + raw)
    ws = wb.active

    if("kwd" in wb.sheetnames and "auth" in wb.sheetnames):
        continue

    titles = []
    journal = raw[:-5].replace('_', ' ').upper()
    years = []
    keywords = []
    keywords_tmp = []
    authors = []
    authors_tmp = []
    MAXROW = ws.max_row
    MAXCOL = ws.max_column

    for r in ws.rows:
        # ignore header
        if(r[0].row == 1):
            continue
        # halt loop
        if(r[0].row > MAXROW):
            break
        # read title and year and refresh
        if(isinstance(r[2].value, str)):
            titles.append(r[2].value)
            years.append(r[1].value)
            if(len(keywords_tmp) > 0):
                keywords.append(keywords_tmp)
                keywords_tmp = []
            if(len(authors_tmp) > 0):
                authors.append(authors_tmp)
                authors_tmp = []
        # read keywords
        if(isinstance(r[3].value, str)):
            keywords_tmp.append(r[3].value)
        # read authors
        if(isinstance(r[4].value, str)):
            authors_tmp.append(r[4].value)
    # refresh at last
    keywords.append(keywords_tmp)
    authors.append(authors_tmp)

    # write w/ modification
    wb.create_sheet("kwd")
    wb.create_sheet("auth")
    ws = wb["kwd"]
    ws.cell(row = 1, column = 1).value = 'title'
    ws.cell(row = 1, column = 2).value = 'journal'
    ws.cell(row = 1, column = 3).value = 'year'
    ws.cell(row = 1, column = 4).value = 'keywords'
    ws = wb["auth"]
    ws.cell(row = 1, column = 1).value = 'title'
    ws.cell(row = 1, column = 2).value = 'authors'
    r = 2
    for i in range(len(titles)):
        ws = wb["kwd"]
        ws.cell(row = r, column = 1).value = titles[i]
        ws.cell(row = r, column = 2).value = journal
        ws.cell(row = r, column = 3).value = years[i]
        for j in range(len(keywords[i])):
            ws.cell(row = r, column = 4 + j).value = keywords[i][j]
        ws = wb["auth"]
        ws.cell(row = r, column = 1).value = titles[i]
        for j in range(len(authors[i])):
            ws.cell(row = r, column = 2 + j).value = authors[i][j]
        r += 1
    
    wb.save(DATAPATH + raw)
    time.sleep(10)
    wb.close()
    
    print("Finished modifying : " + raw)

Finished w/ one
Finished w/ one
Finished w/ one
